In [1]:
import mint
import numpy

In [2]:
regridder = mint.RegridEdges()

# grid settings
regridder.setSrcGridFlags(fixLonAcrossDateline=0, averageLonAtPole=0)
regridder.setDstGridFlags(fixLonAcrossDateline=1, averageLonAtPole=1)

# load the grids
regridder.loadSrcGrid('../../data/latlon100x50.nc$latlon')
regridder.loadDstGrid('../../data/lfric_diag_wind.nc$Mesh2d')

# compute the regridding weights
regridder.build(numCellsPerBucket=1, periodX=360., debug=1)

mint.printLogMessages()
mint.writeLogMessages('log.txt')

In [5]:
!  log.txt

[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [0] total t of segment: 0.000000 != 1 (diff=-1) dst cell 0 points (-45.0000000000000000, 30.1269168853759766), (-39.3750000000000000, 32.3907661437988281)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [1] total t of segment: 0.000000 != 1 (diff=-1) dst cell 2 points (-33.7500000000000000, 34.3084335327148438), (-28.1250000000000000, 35.8960456848144531)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [2] total t of segment: 0.000000 != 1 (diff=-1) dst cell 3 points (-28.1250000000000000, 41.4097557067871094), (-22.5000000000000000, 42.7342109680175781)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [3] total t of segment: 0.000000 != 1 (diff=-1) dst cell 4 points (-22.5000000000000000, 37.1697082519531250), 

[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [1609] total t of segment: 0.000000 != 1 (diff=-1) dst cell 1207 points (72.0122833251953125, 72.3106765747070312), (90.0000000000000000, 73.1250000000000000)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [1610] total t of segment: 0.000000 != 1 (diff=-1) dst cell 1208 points (90.0000000000000000, 67.5000000000000000), (103.3753662109375000, 66.9375305175781250)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [1611] total t of segment: 0.000000 != 1 (diff=-1) dst cell 1208 points (90.0000000000000000, 73.1250000000000000), (107.9877166748046875, 72.3106765747070312)
[Mon Nov  8 14:53:48 2021 Warning ]  src/mntRegridEdges.cpp in function mnt_regridedges_build (line 630): [1612] total t of segment: 0.000000 != 1 (diff=-1) dst cell 1209 points (103.3753662109375000, 66